# Streaming and Interruption - Exercise Notebook

## Objectives
Test your understanding of LangGraph streaming capabilities and how they integrate with interruption patterns by completing the exercises below.

## What You Should Know
- Different streaming modes: `values`, `updates`, and `messages`
- How to stream full state vs. state updates vs. individual messages
- Token-level streaming using `astream_events`
- How streaming works with breakpoints and human-in-the-loop workflows
- Using streaming for real-time feedback and monitoring

## Setup

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai langgraph_sdk

In [ ]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage
from langchain_core.runnables import RunnableConfig
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END, MessagesState
import asyncio
import time

## Exercise 1: Basic Streaming Modes

Create a simple chatbot and explore different streaming modes.

**Your Task:**
1. Create a basic chatbot with conversation summarization
2. Test streaming with `values` mode
3. Test streaming with `updates` mode
4. Compare the outputs and understand the differences

In [ ]:
# Set up the chatbot from the lesson
model = ChatOpenAI(model="gpt-4o", temperature=0)

# Define the extended state class
class State(MessagesState):
    summary: str

# Define call_model function
def call_model(state: State, config: RunnableConfig):
    
    # Get summary if it exists
    summary = state.get("summary", "")

    # If there is summary, then we add it
    if summary:
        
        # Add summary to system message
        system_message = f"Summary of conversation earlier: {summary}"

        # Append summary to any newer messages
        messages = [SystemMessage(content=system_message)] + state["messages"]
    
    else:
        messages = state["messages"]
    
    response = model.invoke(messages, config)
    return {"messages": response}

# Define summarize_conversation function
def summarize_conversation(state: State):
    
    # First, we get any existing summary
    summary = state.get("summary", "")

    # Create our summarization prompt 
    if summary:
        
        # A summary already exists
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )
        
    else:
        summary_message = "Create a summary of the conversation above:"

    # Add prompt to our history
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    
    # Delete all but the 2 most recent messages
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}

# Define should_continue function  
def should_continue(state: State):
    """Return the next node to execute."""
    
    messages = state["messages"]
    
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"
    
    # Otherwise we can just end
    return END

# Build the graph
workflow = StateGraph(State)
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

# Set the entrypoint as conversation
workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)

print("Chatbot created successfully!")

## Exercise 2: Compare Streaming Modes

Test the same conversation with different streaming modes and analyze the differences.

In [ ]:
# Test streaming with 'updates' mode
def test_updates_mode():
    print("=== Testing 'updates' mode ===")
    config = {"configurable": {"thread_id": "updates_test"}}
    
    # Stream with updates mode - shows only state updates after each node
    for chunk in graph.stream({"messages": [HumanMessage(content="Hello! Tell me about Python programming.")]}, config, stream_mode="updates"):
        # Each chunk contains the node name as key and updated state as value
        for node_name, state_update in chunk.items():
            print(f"Node '{node_name}' updated:")
            if "messages" in state_update:
                # Print the new message that was added
                state_update["messages"].pretty_print()
            if "summary" in state_update:
                print(f"Summary updated: {state_update['summary']}")
            print("-" * 40)

# Test streaming with 'values' mode  
def test_values_mode():
    print("\n=== Testing 'values' mode ===")
    config = {"configurable": {"thread_id": "values_test"}}
    
    # Stream with values mode - shows full state after each node
    for state in graph.stream({"messages": [HumanMessage(content="Hello! Tell me about Python programming.")]}, config, stream_mode="values"):
        print(f"Full state contains {len(state['messages'])} messages:")
        # Print last message to show progress
        if state["messages"]:
            state["messages"][-1].pretty_print()
        if state.get("summary"):
            print(f"Current summary: {state['summary'][:100]}...")
        print("=" * 60)

# Test both modes
test_input = {"messages": [HumanMessage(content="Hello! Tell me about Python programming.")]}

# Run both tests and compare the outputs
print("COMPARISON: 'updates' mode shows only what changed in each step")
print("'values' mode shows the complete state after each step\n")

test_updates_mode()
test_values_mode()

## Exercise 3: Token-Level Streaming

Implement token-level streaming to see AI responses as they're generated.

In [ ]:
# Implement token streaming function
async def stream_tokens(graph, input_message, config, node_to_stream='conversation'):
    """
    Stream tokens from a specific node in the graph
    """
    print(f"Streaming tokens from node: {node_to_stream}")
    print("Response: ", end="", flush=True)
    
    # Use astream_events to get token-level streaming
    # Filter for 'on_chat_model_stream' events from the specified node
    async for event in graph.astream_events(input_message, config, version="v2"):
        # Check if this is a chat model stream event from our target node
        if (event["event"] == "on_chat_model_stream" and 
            event['metadata'].get('langgraph_node', '') == node_to_stream):
            # Extract and print the token content
            token_content = event["data"]["chunk"].content
            print(token_content, end="", flush=True)
    
    print()  # New line after streaming

# Test token streaming
async def test_token_streaming():
    config = {"configurable": {"thread_id": "token_test"}}
    input_message = {"messages": [HumanMessage(content="Explain machine learning in simple terms")]}
    
    # Call your streaming function
    await stream_tokens(graph, input_message, config)
    
    print("\n" + "="*50)
    print("Token streaming complete! Notice how you saw each token as it was generated.")

# Run the async test
await test_token_streaming()

## Exercise 4: Streaming with Breakpoints

Combine streaming with breakpoints to create an interactive experience.

In [ ]:
from langgraph.prebuilt import tools_condition, ToolNode

# Define some tools for the agent
def get_weather(location: str) -> str:
    """Get weather information for a location."""
    return f"The weather in {location} is sunny and 75°F."

def calculate(expression: str) -> str:
    """Calculate a mathematical expression."""
    try:
        result = eval(expression)  # Note: eval is unsafe in production!
        return f"The result of {expression} is {result}"
    except:
        return f"Could not calculate {expression}"

# Create an agent with breakpoints and streaming
tools = [get_weather, calculate]
llm_with_tools = model.bind_tools(tools)

def agent_with_tools(state: MessagesState):
    # Call the LLM with tools
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# Build agent graph with breakpoint before tools
agent_builder = StateGraph(MessagesState)
agent_builder.add_node("assistant", agent_with_tools)
agent_builder.add_node("tools", ToolNode(tools))

# Add edges
agent_builder.add_edge(START, "assistant")
agent_builder.add_conditional_edges("assistant", tools_condition)
agent_builder.add_edge("tools", "assistant")

# Compile with breakpoint before tools node
agent_memory = MemorySaver()
agent_graph = agent_builder.compile(checkpointer=agent_memory, interrupt_before=["tools"])

# Function to stream until breakpoint and handle approval
def stream_with_approval(graph, input_msg, thread_config):
    """
    Stream execution until breakpoint, show tool calls, get approval, continue
    """
    print("Starting agent with streaming and approval...")
    
    # Stream until breakpoint
    print("\n--- Streaming until breakpoint ---")
    for chunk in graph.stream(input_msg, thread_config, stream_mode="updates"):
        for node_name, state_update in chunk.items():
            print(f"Node '{node_name}' executed:")
            if "messages" in state_update:
                last_message = state_update["messages"]
                print(f"Message type: {type(last_message).__name__}")
                if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
                    print("Tool calls to be executed:")
                    for tool_call in last_message.tool_calls:
                        print(f"  - {tool_call['name']}: {tool_call['args']}")
                else:
                    last_message.pretty_print()
    
    # Check what tools will be called
    state = graph.get_state(thread_config)
    print(f"\nGraph is interrupted. Next node: {state.next}")
    
    # Simulate user approval (in real app, you'd get user input)
    print("\nSimulating user approval... ✅")
    time.sleep(1)
    
    # Continue execution
    print("\n--- Continuing after approval ---")
    for chunk in graph.stream(None, thread_config, stream_mode="updates"):
        for node_name, state_update in chunk.items():
            print(f"Node '{node_name}' executed:")
            if "messages" in state_update:
                state_update["messages"].pretty_print()
            print("-" * 40)

# Test the streaming + breakpoint combination
test_config = {"configurable": {"thread_id": "agent_test"}}
test_message = {"messages": [HumanMessage(content="What's the weather in San Francisco and calculate 25 * 4?")]}

stream_with_approval(agent_graph, test_message, test_config)

## Exercise 5: Real-time Progress Monitoring

Create a system that provides real-time progress updates during long-running operations.

In [ ]:
import time
from typing_extensions import TypedDict

class ProcessingState(TypedDict):
    task: str
    progress: float  # 0.0 to 1.0
    status: str
    result: str
    steps_completed: list

# Define processing steps that take time
def data_collection(state: ProcessingState) -> ProcessingState:
    """
    Simulate data collection phase
    """
    print("Starting data collection...")
    
    # Simulate time-consuming task with progress updates
    for i in range(3):
        time.sleep(0.5)  # Simulate work
        progress = (i + 1) / 3 * 0.3  # First 30% of total progress
        print(f"  Collecting data... {progress*100:.0f}% complete")
    
    return {
        "task": state["task"],
        "progress": 0.3,
        "status": "Data collection complete",
        "result": state.get("result", ""),
        "steps_completed": state.get("steps_completed", []) + ["data_collection"]
    }

def data_processing(state: ProcessingState) -> ProcessingState:
    """
    Simulate data processing phase
    """
    print("Starting data processing...")
    
    # Simulate processing with multiple steps
    steps = ["Cleaning data", "Analyzing patterns", "Applying algorithms"]
    
    for i, step in enumerate(steps):
        time.sleep(0.3)  # Simulate work
        progress = 0.3 + (i + 1) / len(steps) * 0.5  # 30% + next 50% of total progress
        print(f"  {step}... {progress*100:.0f}% complete")
    
    return {
        "task": state["task"],
        "progress": 0.8,
        "status": "Data processing complete", 
        "result": "Processed 10,000 records with 95% accuracy",
        "steps_completed": state["steps_completed"] + ["data_processing"]
    }

def result_generation(state: ProcessingState) -> ProcessingState:
    """
    Generate final results
    """
    print("Generating final results...")
    time.sleep(0.5)  # Simulate final processing
    
    final_result = f"Task '{state['task']}' completed successfully. {state['result']}. Generated comprehensive report."
    
    return {
        "task": state["task"],
        "progress": 1.0,
        "status": "Complete",
        "result": final_result,
        "steps_completed": state["steps_completed"] + ["result_generation"]
    }

# Build processing pipeline graph
processing_builder = StateGraph(ProcessingState)
processing_builder.add_node("data_collection", data_collection)
processing_builder.add_node("data_processing", data_processing)
processing_builder.add_node("result_generation", result_generation)

# Add edges to create pipeline
processing_builder.add_edge(START, "data_collection")
processing_builder.add_edge("data_collection", "data_processing")
processing_builder.add_edge("data_processing", "result_generation")
processing_builder.add_edge("result_generation", END)

processing_graph = processing_builder.compile()

# Function to stream progress updates
def stream_progress(graph, initial_state, thread_config):
    """
    Stream progress updates in real-time
    """
    print("Starting long-running process...")
    print(f"Task: {initial_state['task']}")
    print("="*60)
    
    # Stream with progress updates
    for chunk in graph.stream(initial_state, thread_config, stream_mode="updates"):
        for node_name, state_update in chunk.items():
            print(f"\n[{node_name.upper()}] {state_update['status']}")
            
            # Show progress bar
            progress = state_update['progress']
            bar_length = 30
            filled_length = int(bar_length * progress)
            bar = '█' * filled_length + '░' * (bar_length - filled_length)
            print(f"Progress: [{bar}] {progress*100:.1f}%")
            
            # Show completed steps
            if state_update['steps_completed']:
                print(f"Completed steps: {', '.join(state_update['steps_completed'])}")
            
            # Show result if available
            if state_update.get('result') and state_update['result']:
                print(f"Current result: {state_update['result']}")
            
            print("-" * 40)

# Test progress monitoring
initial_state = {
    "task": "Machine Learning Model Training",
    "progress": 0.0,
    "status": "Starting",
    "result": "",
    "steps_completed": []
}

thread_config = {"configurable": {"thread_id": "progress_test"}}
stream_progress(processing_graph, initial_state, thread_config)

## Exercise 6: Custom Streaming Event Handler

Create a custom event handler that processes different types of streaming events.

In [ ]:
class StreamEventHandler:
    def __init__(self):
        self.message_count = 0
        self.token_count = 0
        self.event_log = []
        self.nodes_executed = set()
        self.total_execution_time = 0
        self.start_time = None
    
    def handle_event(self, event):
        """
        Handle different types of streaming events
        """
        event_type = event.get('event', 'unknown')
        timestamp = time.time()
        
        # Initialize start time on first event
        if self.start_time is None:
            self.start_time = timestamp
        
        # Handle different event types
        if event_type == 'on_chat_model_stream':
            # Handle token streaming
            token = event['data'].get('chunk', {})
            if hasattr(token, 'content') and token.content:
                self.token_count += 1
                print(token.content, end='', flush=True)
                
        elif event_type == 'on_chain_start':
            # Handle chain start events
            node_name = event['metadata'].get('langgraph_node', 'unknown')
            if node_name != 'unknown':
                self.nodes_executed.add(node_name)
                print(f"\n🚀 Starting node: {node_name}")
                
        elif event_type == 'on_chain_end':
            # Handle chain end events
            node_name = event['metadata'].get('langgraph_node', 'unknown')
            if node_name != 'unknown':
                print(f"\n✅ Completed node: {node_name}")
                
        elif event_type == 'on_chat_model_start':
            # Handle LLM call start
            print(f"\n🤖 Starting LLM call...")
            
        elif event_type == 'on_chat_model_end':
            # Handle LLM call end
            print(f"\n✨ LLM call complete")
            self.message_count += 1
            
        # Log all events for analysis
        self.event_log.append({
            'event': event_type,
            'timestamp': timestamp,
            'node': event['metadata'].get('langgraph_node', 'unknown'),
            'data_keys': list(event.get('data', {}).keys())
        })
    
    def get_statistics(self):
        """
        Return streaming statistics
        """
        end_time = time.time()
        execution_time = end_time - self.start_time if self.start_time else 0
        
        return {
            "messages_processed": self.message_count,
            "tokens_streamed": self.token_count,
            "total_events": len(self.event_log),
            "nodes_executed": list(self.nodes_executed),
            "execution_time_seconds": round(execution_time, 2),
            "events_per_second": round(len(self.event_log) / execution_time, 2) if execution_time > 0 else 0
        }

# Test custom event handler
async def test_custom_handler():
    handler = StreamEventHandler()
    config = {"configurable": {"thread_id": "handler_test"}}
    
    print("Testing custom event handler...")
    print("="*50)
    
    # Use your handler with astream_events
    async for event in graph.astream_events(
        {"messages": [HumanMessage(content="Write a short story about a robot learning to paint")]},
        config,
        version="v2"
    ):
        handler.handle_event(event)
    
    # Show final statistics
    stats = handler.get_statistics()
    print("\n\n" + "="*50)
    print("STREAMING STATISTICS:")
    print("="*50)
    for key, value in stats.items():
        print(f"{key.replace('_', ' ').title()}: {value}")
    
    # Show event type breakdown
    event_types = {}
    for event_info in handler.event_log:
        event_type = event_info['event']
        event_types[event_type] = event_types.get(event_type, 0) + 1
    
    print(f"\nEvent Type Breakdown:")
    for event_type, count in sorted(event_types.items()):
        print(f"  {event_type}: {count}")

# Run the test
await test_custom_handler()

## Exercise 7: Streaming Performance Analysis

Analyze the performance characteristics of different streaming approaches.

In [ ]:
import time
import asyncio

async def benchmark_streaming_modes():
    """
    Benchmark different streaming modes for performance
    """
    test_input = {"messages": [HumanMessage(content="Write a detailed explanation of quantum computing")]}
    
    print("🚀 STREAMING PERFORMANCE BENCHMARK")
    print("="*60)
    
    # Benchmark 'values' mode
    print("Benchmarking 'values' mode...")
    config_values = {"configurable": {"thread_id": "benchmark_values"}}
    start_time = time.time()
    
    # Run with values mode and measure time
    chunk_count = 0
    for chunk in graph.stream(test_input, config_values, stream_mode="values"):
        chunk_count += 1
        # Don't print to avoid cluttering output
        
    values_time = time.time() - start_time
    print(f"  ✅ Completed in {values_time:.2f}s with {chunk_count} chunks")
    
    # Benchmark 'updates' mode
    print("\nBenchmarking 'updates' mode...")
    config_updates = {"configurable": {"thread_id": "benchmark_updates"}}
    start_time = time.time()
    
    # Run with updates mode and measure time
    chunk_count = 0
    for chunk in graph.stream(test_input, config_updates, stream_mode="updates"):
        chunk_count += 1
        
    updates_time = time.time() - start_time
    print(f"  ✅ Completed in {updates_time:.2f}s with {chunk_count} chunks")
    
    # Benchmark token streaming
    print("\nBenchmarking token streaming...")
    config_tokens = {"configurable": {"thread_id": "benchmark_tokens"}}
    start_time = time.time()
    
    # Run with token streaming and measure time
    event_count = 0
    async for event in graph.astream_events(test_input, config_tokens, version="v2"):
        event_count += 1
        # Count all events but don't process to get pure overhead
        
    token_time = time.time() - start_time
    print(f"  ✅ Completed in {token_time:.2f}s with {event_count} events")
    
    # Report results
    print("\n" + "="*60)
    print("📊 PERFORMANCE RESULTS")
    print("="*60)
    
    results = [
        ("Values mode", values_time, "Full state after each node"),
        ("Updates mode", updates_time, "Only state changes"),
        ("Token streaming", token_time, "All events including tokens")
    ]
    
    # Sort by performance
    results.sort(key=lambda x: x[1])
    
    for i, (mode, exec_time, description) in enumerate(results):
        rank = "🥇" if i == 0 else "🥈" if i == 1 else "🥉"
        print(f"{rank} {mode}: {exec_time:.2f}s - {description}")
    
    # Calculate overhead
    baseline = results[0][1]  # Fastest time
    print(f"\nStreaming Overhead Analysis:")
    for mode, exec_time, _ in results:
        overhead = ((exec_time - baseline) / baseline) * 100
        print(f"  {mode}: +{overhead:.1f}% overhead")

# Run benchmark
await benchmark_streaming_modes()

## Reflection Questions

Answer these questions to test your understanding:

1. **What's the difference between 'values' and 'updates' streaming modes?**
   - Your answer: Values mode streams the complete state after each node executes, while updates mode streams only the changes/updates made by each node. Values mode gives you full context but more data, updates mode is more efficient but only shows what changed.

2. **When would you use token-level streaming vs. state-level streaming?**
   - Your answer: Token-level streaming is ideal for real-time chat interfaces where you want to show AI responses as they're generated, providing immediate user feedback. State-level streaming is better for monitoring graph execution progress, debugging workflows, or when you need to see how data flows through your pipeline.

3. **How does streaming work with breakpoints and human-in-the-loop workflows?**
   - Your answer: Streaming allows you to monitor execution up to a breakpoint, see what operations are about to be performed, get user approval or input, then continue streaming the remaining execution. This provides transparency into the decision-making process before critical operations.

4. **What are the performance implications of different streaming modes?**
   - Your answer: Updates mode is generally most efficient (least data transferred), values mode has moderate overhead (full state each time), and token-level streaming has the highest overhead (many small events) but provides the richest real-time experience. Choose based on your use case requirements.

5. **How can streaming be used for real-time monitoring and user feedback?**
   - Your answer: Streaming enables progress bars, real-time status updates, live chat responses, system monitoring dashboards, error detection during execution, and interactive debugging. It transforms batch operations into observable, interactive processes.

## Key Takeaways

After completing this notebook, you should understand:

✅ **Different streaming modes serve different purposes:**
   - `values`: Full state monitoring and debugging
   - `updates`: Efficient change tracking
   - `events`: Token-level real-time feedback

✅ **Streaming integrates seamlessly with human-in-the-loop patterns:**
   - Monitor execution until breakpoints
   - Provide transparent decision-making
   - Enable interactive approval workflows

✅ **Performance considerations matter:**
   - Choose the right streaming mode for your use case
   - Balance real-time feedback with system efficiency
   - Consider network and processing overhead

✅ **Custom event handlers enable sophisticated monitoring:**
   - Process different event types appropriately
   - Maintain metrics and analytics
   - Create interactive dashboards and controls

## Challenge: Multi-Modal Streaming Dashboard

**Advanced Exercise:** Create a comprehensive streaming dashboard that demonstrates mastery of all streaming concepts. Your dashboard should:

### Core Requirements:
- ✅ **Multi-Stream Management**: Handle multiple concurrent graph executions
- ✅ **Real-Time Updates**: Display live progress across different streaming modes  
- ✅ **Interactive Controls**: Pause, resume, cancel operations
- ✅ **Performance Analytics**: Track metrics and identify bottlenecks
- ✅ **Error Handling**: Gracefully handle failures and provide recovery options

### Advanced Features:
- 🎯 **Smart Filtering**: Show/hide specific event types or nodes
- 📊 **Visual Analytics**: Generate performance charts and statistics
- 🔍 **Search & Replay**: Search through event history and replay executions
- 🚨 **Alerting System**: Notify when streams exceed time thresholds
- 💾 **Export Capabilities**: Save execution logs and metrics to files

### Extension Ideas:
1. **Historical Analysis**: Compare performance across multiple runs
2. **Resource Monitoring**: Track memory and CPU usage during streaming
3. **Custom Visualizations**: Create graphs showing token generation rates
4. **Integration Testing**: Test streaming with different LLM providers
5. **Load Testing**: Simulate high-volume concurrent streaming scenarios

This challenge combines all the concepts from this notebook into a real-world application that could be used for monitoring production LangGraph systems!

In [ ]:
class StreamingDashboard:
    def __init__(self):
        self.active_streams = {}
        self.metrics = {}
        self.event_history = []
        self.user_interactions = []
        self.dashboard_active = False
    
    async def start_stream_monitoring(self, graph, input_data, stream_id, stream_mode="updates"):
        """
        Start monitoring a graph stream with a unique ID
        """
        print(f"🎯 Starting stream monitoring for: {stream_id}")
        self.active_streams[stream_id] = {
            "status": "running",
            "start_time": time.time(),
            "messages_count": 0,
            "current_node": None,
            "progress": 0.0
        }
        
        config = {"configurable": {"thread_id": f"dashboard_{stream_id}"}}
        
        try:
            if stream_mode == "events":
                # Monitor with detailed events
                async for event in graph.astream_events(input_data, config, version="v2"):
                    await self._process_event(stream_id, event)
                    
            else:
                # Monitor with state updates
                for chunk in graph.stream(input_data, config, stream_mode=stream_mode):
                    await self._process_chunk(stream_id, chunk)
                    
        except Exception as e:
            self.active_streams[stream_id]["status"] = f"error: {e}"
        else:
            self.active_streams[stream_id]["status"] = "completed"
            self.active_streams[stream_id]["end_time"] = time.time()
        
        print(f"✅ Stream {stream_id} completed")
    
    async def _process_event(self, stream_id, event):
        """Process individual events from astream_events"""
        event_type = event.get('event', 'unknown')
        node_name = event['metadata'].get('langgraph_node', 'unknown')
        
        # Update stream info
        if stream_id in self.active_streams:
            self.active_streams[stream_id]["current_node"] = node_name
            
        # Log event
        self.event_history.append({
            "stream_id": stream_id,
            "event_type": event_type,
            "node": node_name,
            "timestamp": time.time()
        })
        
        # Show real-time progress
        if event_type == "on_chat_model_stream":
            token = event['data'].get('chunk', {})
            if hasattr(token, 'content') and token.content:
                print(token.content, end='', flush=True)
    
    async def _process_chunk(self, stream_id, chunk):
        """Process chunks from regular streaming"""
        for node_name, state_update in chunk.items():
            if stream_id in self.active_streams:
                self.active_streams[stream_id]["current_node"] = node_name
                if "messages" in state_update:
                    self.active_streams[stream_id]["messages_count"] += 1
    
    def display_dashboard(self):
        """
        Display the current dashboard state
        """
        print("\n" + "="*80)
        print("🖥️  LANGGRAPH STREAMING DASHBOARD")
        print("="*80)
        
        if not self.active_streams:
            print("No active streams")
            return
        
        # Display active streams
        for stream_id, info in self.active_streams.items():
            status_emoji = "🔄" if info["status"] == "running" else "✅" if info["status"] == "completed" else "❌"
            
            print(f"{status_emoji} Stream: {stream_id}")
            print(f"   Status: {info['status']}")
            print(f"   Current Node: {info.get('current_node', 'N/A')}")
            print(f"   Messages: {info.get('messages_count', 0)}")
            
            if "start_time" in info:
                elapsed = time.time() - info["start_time"]
                print(f"   Runtime: {elapsed:.1f}s")
                
            print("-" * 40)
        
        # Display recent events
        if self.event_history:
            print(f"\n📊 Recent Events (last 5):")
            for event in self.event_history[-5:]:
                print(f"   {event['event_type']} in {event['node']} ({event['stream_id']})")
        
        # Display metrics
        total_events = len(self.event_history)
        active_count = sum(1 for s in self.active_streams.values() if s["status"] == "running")
        completed_count = sum(1 for s in self.active_streams.values() if s["status"] == "completed")
        
        print(f"\n📈 Metrics:")
        print(f"   Total Events: {total_events}")
        print(f"   Active Streams: {active_count}")
        print(f"   Completed Streams: {completed_count}")
    
    def handle_user_interaction(self, action, stream_id=None):
        """
        Handle user interactions with the dashboard
        """
        timestamp = time.time()
        
        if action == "pause" and stream_id:
            # In real implementation, would pause the specific stream
            print(f"⏸️ Pausing stream {stream_id}")
            
        elif action == "resume" and stream_id:
            print(f"▶️ Resuming stream {stream_id}")
            
        elif action == "cancel" and stream_id:
            print(f"❌ Cancelling stream {stream_id}")
            if stream_id in self.active_streams:
                self.active_streams[stream_id]["status"] = "cancelled"
                
        elif action == "refresh":
            self.display_dashboard()
            
        # Log interaction
        self.user_interactions.append({
            "action": action,
            "stream_id": stream_id,
            "timestamp": timestamp
        })

# Implement a comprehensive streaming dashboard test
async def test_streaming_dashboard():
    """
    Test the streaming dashboard with multiple concurrent operations
    """
    dashboard = StreamingDashboard()
    
    print("🚀 Testing Multi-Stream Dashboard")
    print("="*60)
    
    # Start multiple concurrent streams
    tasks = []
    
    # Stream 1: Simple conversation
    task1 = asyncio.create_task(
        dashboard.start_stream_monitoring(
            graph, 
            {"messages": [HumanMessage(content="Tell me about artificial intelligence")]},
            "ai_chat",
            "updates"
        )
    )
    tasks.append(task1)
    
    # Stream 2: Agent with tools
    task2 = asyncio.create_task(
        dashboard.start_stream_monitoring(
            agent_graph,
            {"messages": [HumanMessage(content="What's 15 * 23?")]},
            "math_agent", 
            "updates"
        )
    )
    tasks.append(task2)
    
    # Stream 3: Processing pipeline  
    task3 = asyncio.create_task(
        dashboard.start_stream_monitoring(
            processing_graph,
            {
                "task": "Data Analysis Pipeline",
                "progress": 0.0,
                "status": "Starting",
                "result": "",
                "steps_completed": []
            },
            "data_pipeline",
            "updates"
        )
    )
    tasks.append(task3)
    
    # Monitor progress
    for i in range(5):  # Check status 5 times
        await asyncio.sleep(2)  # Wait 2 seconds
        print(f"\n⏰ Status Update #{i+1}")
        dashboard.display_dashboard()
    
    # Simulate user interaction
    dashboard.handle_user_interaction("refresh")
    
    # Wait for all tasks to complete
    await asyncio.gather(*tasks)
    
    # Final dashboard state
    print("\n🏁 Final Dashboard State:")
    dashboard.display_dashboard()
    
    # Export metrics
    print(f"\n📤 Session Summary:")
    print(f"   Total streams processed: {len(dashboard.active_streams)}")
    print(f"   Total events captured: {len(dashboard.event_history)}")
    print(f"   User interactions: {len(dashboard.user_interactions)}")

# Run the comprehensive dashboard test
await test_streaming_dashboard()